In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 환경 설정

In [ ]:
# 위까지 통합코드 (GPT)

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip3 install kobert-transformers

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_transformers import get_kobert_model, get_tokenizer

device = torch.device("cuda:0")

model = get_kobert_model()
tokenizer = get_tokenizer() # vocab 역할 

# 이후부터 모델 및 토크나이저를 사용하여 작업을 진행할 수 있습니다.


# 데이터셋 전처리

In [64]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/KDT/Final Project/data/BGZT_sample_data.csv')

In [31]:
data

,product_id,product_name,cat_id
0,223861589,노스페이스 롱패딩,310090050
1,211498059,새상품 뉴발란스 베스트 블랙 패딩,310090050
2,210612980,정품 빈폴 구스다운,310090050
3,223417908,새상품급 정품 얇은 고급 패딩 착샷 통통,310090050
4,211768829,네파 롱패딩 팝니다 가격내림,310090050
...,...,...,...
57094,174798546,방울털모자 겨울모자,400070999
57095,164895194,모자,400070999
57096,220016723,노스페이스 남성등산모자 고어텍스 여름등산모자 사이즈,400070999
57097,113371163,모자,400070999


In [49]:
len(data['cat_id'].unique())

146

In [17]:
data['cat_id'].unique()

array([310090050, 310090060, 310090020, 310090999, 310110030, 310110010,
       310260200, 310260100, 310260300, 310040090, 310040080, 310040070,
       310040999, 310050010, 310050050, 310050030, 310050040, 310050999,
       310150030, 310150010, 310150040, 310150070, 310150999, 310140020,
       310140010, 310140030, 310140040, 310140999, 310160010, 310160060,
       310160020, 310160999, 310130030, 310130080, 310130040, 310120030,
       310120110, 310120020, 310100040, 310100020, 310100030, 310100060,
       310100050, 310100999, 310180030, 310180060, 310180010, 310180999,
       310080010, 310080050, 310080020, 310080040, 310080030, 310080999,
       310190010, 310190020, 310190030, 310190999, 320080700, 320080800,
       320080600, 320080999, 320100100, 320100200, 320210200, 320210100,
       320210300, 320040100, 320040200, 320040500, 320040400, 320040999,
       320120100, 320120200, 320120300, 320120999, 320110100, 320110200,
       320110600, 320110500, 320110999, 320130100, 

In [66]:
data['cat_id'] = data['cat_id'].astype('str')

In [53]:
type(data['cat_id'][0])

str

In [67]:
# !pip install sklearn
from sklearn.preprocessing import LabelEncoder
label = ['cat_id']
data[label] = data[label].apply(LabelEncoder().fit_transform)

In [8]:
data

,product_id,product_name,cat_id
0,223861589,노스페이스 롱패딩,16
1,211498059,새상품 뉴발란스 베스트 블랙 패딩,16
2,210612980,정품 빈폴 구스다운,16
3,223417908,새상품급 정품 얇은 고급 패딩 착샷 통통,16
4,211768829,네파 롱패딩 팝니다 가격내림,16
...,...,...,...
57094,174798546,방울털모자 겨울모자,112
57095,164895194,모자,112
57096,220016723,노스페이스 남성등산모자 고어텍스 여름등산모자 사이즈,112
57097,113371163,모자,112


In [9]:
len(data['cat_id'].unique())

146

In [10]:
data[data['cat_id'] == 0]

,product_id,product_name,cat_id
4400,222664067,오즈세컨정품,0
4401,196402541,마른사이즈 언발란스나시 킹콩빈티지,0
4402,124155565,수입 플리츠 민소매 블라우스,0
4403,189217427,새상품 민소매 블라우스,0
4404,224965873,케네스레이디 레이스 민소매 슬리브리스 블라우스 셔츠 살구색 베이지 연어색,0
...,...,...,...
4795,137404727,옷개 전부 만원 치마 오프숄더 블라우스 치마바지,0
4796,193315646,린넨 리본 나시 브라운 새상품,0
4797,224011947,쉬폰 데님 세트,0
4798,151339993,민소매 블라우스,0


In [68]:
data_list = []
for q, label in zip(data['product_name'], data['cat_id'])  :
    data1 = []
    data1.append(q)
    data1.append(label)

    data_list.append(data1)

In [ ]:
data_list[0]

['노스페이스 롱패딩', 310090050]

# Train data & Test data

In [69]:
# train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=25)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

45679
11420


# KoBERT 입력 데이터로 만들기

In [80]:
from transformers import BertTokenizer

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        self.sentences = [i[sent_idx] for i in dataset]
        self.labels = [torch.tensor(i[label_idx], dtype=torch.long) for i in dataset]
        self.tokenizer = bert_tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair

    def __getitem__(self, i):
        sentence = self.sentences[i]
        label = self.labels[i]

        inputs = self.tokenizer.encode_plus(
            sentence,
            None if not self.pair else sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length' if self.pad else None,
            truncation=True
        )

        input_ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)
        # segment 밑에서 에러 떠서 추가 
        segment_ids = torch.tensor(inputs['token_type_ids'], dtype=torch.long)
        valid_length = torch.tensor(len(inputs['input_ids']), dtype=torch.long)

        return (input_ids, attention_mask, segment_ids, valid_length, label) # segment 추가

    def __len__(self):
        return len(self.labels)


In [81]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [82]:
# 토큰화
tokenizer = get_tokenizer()
tok = tokenizer

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
data_train[0]

(tensor([   2, 5117, 6228, 5920, 4257, 7431, 6273, 7318,    3,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor(64),
 tensor(310140030))

In [ ]:
dataset_train[0] # [CLS], "화이트", "##무드", "중청", "##바", "##지", [SEP]

['화이트무드 중청바지', 310140030]

In [83]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

# KoBERT 학습모델 만들기 

In [11]:
len(data['cat_id'].unique()) # 카테고리 수 146개

146

In [89]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 146,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate is not None:
            self.dropout = nn.Dropout(p=dr_rate)
        else:
            self.dropout = None
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids, dtype=torch.float32)
        for i, v in enumerate(valid_length):
            # attention_mask[i][:v] = 1 # 학습까진 잘 됐는데 새로운 거 확인해보는 부분에서 에러 나서 주석처리 후 수정.
            attention_mask[i, :v[i]] = 1
        return attention_mask

    # def forward(self, token_ids, valid_length, segment_ids):
    #     attention_mask = self.gen_attention_mask(token_ids, valid_length)
    #     token_ids = token_ids.to(device)
    #     valid_length = valid_length.to(device)
    #     segment_ids = segment_ids.to(device)

    #     # print(token_ids.dtype)
    #     # print(valid_length.dtype)
    #     # print(attention_mask.dtype)
        
    #     pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    #     pooler = pooler[1].to(device)
    #     if self.dropout is not None:
    #         out = self.dropout(pooler)
    #         return self.classifier(out)
    #     else: 
    #         return self.classifier(pooler)

    def forward(self, input_ids, attention_mask, segment_ids, valid_length):
        _, pooled_output = self.bert(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    token_type_ids=segment_ids,
                                    return_dict=False)
        output = self.dropout(pooled_output)
        output = self.classifier(output)
        return output


In [70]:
# ## cpu로 바꿔서 진행 

# class BERTClassifier(nn.Module):
#     def __init__(self,
#                  bert,
#                  hidden_size = 768,
#                  num_classes = 146,   ##클래스 수 조정##
#                  dr_rate=None,
#                  params=None):
#         super(BERTClassifier, self).__init__()
#         self.bert = bert
#         self.dr_rate = dr_rate
                 
#         self.classifier = nn.Linear(hidden_size, num_classes)
#         if dr_rate is not None:
#             self.dropout = nn.Dropout(p=dr_rate)
#         else:
#             self.dropout = None
    
#     def gen_attention_mask(self, token_ids, valid_length):
#         attention_mask = torch.zeros_like(token_ids, dtype=torch.float32)
#         for i, v in enumerate(valid_length):
#             attention_mask[i][:v] = 1
#         return attention_mask

#     def forward(self, token_ids, valid_length, segment_ids):
#         attention_mask = self.gen_attention_mask(token_ids, valid_length)
#         token_ids = token_ids.cpu()
#         valid_length = valid_length.cpu()
#         segment_ids = segment_ids.cpu()

#         print(token_ids.dtype)
#         print(valid_length.dtype)
#         print(attention_mask.dtype)

#         pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device))
#         pooler = pooler[1].cpu()
#         if self.dropout is not None:
#             out = self.dropout(pooler)
#             return self.classifier(out)
#         else:
#             return self.classifier(pooler)


In [ ]:
# GPU 

from torch.optim import AdamW

# BERT 모델 불러오기
kobert_model = get_kobert_model() 
model = BERTClassifier(kobert_model,  dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
optimizer = AdamW(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    

In [90]:
## cpu로 진행 

# BERT 모델 불러오기
kobert_model = get_kobert_model() 
model = BERTClassifier(kobert_model,  dr_rate=0.5)
# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X, Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices.cpu() == Y.cpu()).sum().data.numpy() / max_indices.size()[0]
    return train_acc


# KoBERT 모델 학습시키기

In [ ]:
from tqdm.notebook import tqdm

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        attention_mask = attention_mask.long().to(device)
        valid_length = valid_length.to(device)
        label = label.long().to(device)
        out = model(token_ids, valid_length, attention_mask)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        attention_mask = attention_mask.long().to(device)
        valid_length = valid_length.to(device)
        label = label.long().to(device)
        out = model(token_ids, valid_length, attention_mask)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


In [ ]:
# ## cpu

# from tqdm.notebook import tqdm

# device = torch.device('cpu')  # CPU 장치로 변경
# model.to(device)

# for e in range(num_epochs):
#     train_acc = 0.0
#     test_acc = 0.0
#     model.train()
#     for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(train_dataloader)):
#         optimizer.zero_grad()
#         token_ids = token_ids.long().to(device)
#         attention_mask = attention_mask.long().to(device)
#         valid_length = valid_length.to(device)
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, attention_mask)
#         loss = loss_fn(out, label)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
#         optimizer.step()
#         scheduler.step()  # Update learning rate schedule
#         train_acc += calc_accuracy(out, label)
#         if batch_id % log_interval == 0:
#             print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
#     print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
#     model.eval()
#     for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(test_dataloader)):
#         token_ids = token_ids.long().to(device)
#         attention_mask = attention_mask.long().to(device)
#         valid_length = valid_length.to(device)
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, attention_mask)
#         test_acc += calc_accuracy(out, label)
#     print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


In [76]:
# 모델 저장 경로 지정
save_path = "/content/drive/MyDrive/KDT/Final Project/modeling/model.pth"

# 모델 저장
torch.save(model.state_dict(), save_path)

In [91]:
# 저장된 모델 불러오기
# model = BERTClassifier(kobert_model, dr_rate=0.5).to(device)
# save_path = "/content/drive/MyDrive/KDT/Final Project/modeling/model_kobert_epoch5.pth"
# model.load_state_dict(torch.load(save_path))


device = torch.device("cpu")
model = BERTClassifier(kobert_model, dr_rate=0.5)
save_path = "/content/drive/MyDrive/KDT/Final Project/modeling/model_kobert_epoch5.pth"
model.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))


<All keys matched successfully>

# 새로운 문장 테스트

In [92]:
import numpy as np
from kobert_transformers import get_tokenizer

tokenizer = get_tokenizer()
tok = get_tokenizer()

# 카테고리와 인덱스 매핑
category_map = {}
for i in range(146):
    category_map[i] = f"카테고리{i+1}"

def predict(predict_sentence):
    category_id = 0  # 예측할 카테고리의 인덱스 값
    data = [predict_sentence, category_id]  # 상품명, 카테고리id
    print(data)
    dataset_another = [data]
    print(dataset_another)

#                          dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
# data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)

    another_test = BERTDataset(dataset_another, sent_idx=0, label_idx=1, bert_tokenizer=tok, max_len=max_len, pad=True, pair=False)  # 인자 수정

    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)

    model.eval()

    # for batch_id, (token_ids, valid_length, label) in enumerate(test_dataloader):
    #     token_ids = token_ids.long().to(device)
    #     #segment_ids = segment_ids.long().to(device)
    #     valid_length = valid_length.to(device)
    #     label = label.long().to(device)

    #     out = model(token_ids, valid_length, segment_ids=torch.zeros_like(token_ids))
 
    for batch_id, (input_ids, attention_mask, segment_ids, valid_length, label) in enumerate(test_dataloader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        segment_ids = segment_ids.to(device)
        valid_length = valid_length.to(device)
        label = label.to(device)

        out = model(input_ids, valid_length, segment_ids, attention_mask)

        test_eval = []
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) in category_map:
                category = category_map[np.argmax(logits)]
                test_eval.append(category)

        if test_eval:
            print(">> 입력하신 상품명으로부터 예측된 카테고리:", test_eval[0])
        else:
            print(">> 카테고리를 예측할 수 없습니다.")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [93]:
# 질문 무한반복하기! 0 입력시 종료
while True:
    sentence = input("상품명을 입력해주세요 : ")
    if sentence == '0':
        break
    predict(sentence)
    print("\n")

상품명을 입력해주세요 : 모자
['모자', 0]
[['모자', 0]]


ValueError: ignored